In [1]:
#Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#Guardar modelos
from scipy.sparse import csr_matrix
from scipy import sparse


#LDA
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from pprint import pprint

# Carga

## Cargar datos

In [2]:
file_path_meta = "https://raw.githubusercontent.com/juankquintana/aprendizaje-no-supervisado/main/Proyecto/data/movies_metadata.csv"

metadata = pd.read_csv(file_path_meta)

metadata.head(1)

C:\Users\WD\AppData\Local\Temp\ipykernel_15644\834555082.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(file_path_meta)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [3]:
file_path_usuarios = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/base_usuarios.csv"

df_usuarios = pd.read_csv(file_path_usuarios)

df_usuarios = df_usuarios.drop(df_usuarios.columns[0], axis=1)

df_usuarios.head(5)

,userId,movieId,rating
0,123687,163,4.0
1,221195,4973,4.0
2,134366,1643,4.0
3,20721,3034,3.5
4,225569,165,5.0


In [4]:
df_keywords_title = pd.read_pickle('ModelosNLP/df_keywords_title.pkl')
df_keywords_title.head(5)

,id,original_title,keyword_name,cleaned_keywords,genre_names
0,2,Ariel,"[underdog, prison, factory worker, prisoner, h...","[Drama, Crime, underdog, prison, factori worke...","[Drama, Crime]"
1,3,Varjoja paratiisissa,"[salesclerk, helsinki, garbage, independent film]","[Drama, Comedy, salesclerk, helsinki, garbag, ...","[Drama, Comedy]"
2,5,Four Rooms,"[hotel, new year's eve, witch, bet, hotel room...","[Crime, Comedy, hotel, new year eve, witch, be...","[Crime, Comedy]"
3,6,Judgment Night,"[chicago, drug dealer, boxing match, escape, o...","[Action, Thriller, Crime, chicago, drug dealer...","[Action, Thriller, Crime]"
4,11,Star Wars,"[android, galaxy, hermit, death star, lightsab...","[Adventure, Action, Science Fiction, android, ...","[Adventure, Action, Science Fiction]"


In [5]:
df_usuarios['movieId']=df_usuarios['movieId'].astype(str)

df_merge = pd.merge(df_usuarios, df_keywords_title, left_on='movieId', right_on='id', how='inner')

# Seleccionamos las columnas necesarias: userId, movieId, original_title y rating.
df_ratings_peliculas = df_merge[['userId', 'movieId', 'original_title', 'rating']]

# Mostramos el dataframe resultante.
print(df_ratings_peliculas)

         userId movieId                            original_title  rating
0        123687     163                            Ocean's Twelve     4.0
1        203854     163                            Ocean's Twelve     5.0
2         78563     163                            Ocean's Twelve     1.0
3        203732     163                            Ocean's Twelve     4.0
4        195546     163                            Ocean's Twelve     5.0
...         ...     ...                                       ...     ...
2275627  100838  102197                                     Шпион     3.0
2275628  243443  169314                  Where The Dead Go to Die     0.5
2275629   45811  126777                            If I Were King     3.0
2275630  228291   84111  Na xie nian, wo men yi qi zhui de nu hai     4.0
2275631  111274  137182               The Broken Circle Breakdown     4.5

[2275632 rows x 4 columns]


In [ ]:
#Cargar datos para recomendacion por genero
user_gener_c=pd.read_csv('data/user_gener_c.csv')

## Cargar funciones de recomendacion

### Cargar Similitud de Coseno con Count Vectorizer

In [7]:
from scipy import sparse
cosine_sim_sparseCOUNT = sparse.load_npz('ModelosNLP/cosine_sim_sparse_matrixCOUNT.npz')

# Convertirla de nuevo aU densa (si es necesario, aunque puedes seguir usándola como dispersa)
cosine_sim_sparseCOUNT = cosine_sim_sparseCOUNT.toarray()


### Recomendador Coseno

In [8]:
def recomendador_coseno(movie_id, cosine_sim=cosine_sim_sparseCOUNT, num_recomendaciones=5, df=df_keywords_title):
    # Crear una Serie que contiene como índice el 'id' y el valor correspondiente al índice de la película
    indices = pd.Series(df.index, index=df['id'])

    # Verificar si el movie_id está en los índices
    if movie_id not in indices.index:
        print(f'ID de película {movie_id} no encontrado.')
        return

    # Obtener el índice de la película a partir del movie_id
    idx = indices[movie_id]

    # Obtener el título de la película original
    original_title = df.loc[idx, 'original_title']
    #print(f'Título de la película original: {original_title}')
    #print(f'Índice de la película: {movie_id}')

    # Obtener la similitud de coseno para esa película en particular con todas las películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar la lista de tuplas por la similitud de coseno en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las películas más similares ignorando el primero (que es la misma película)
    sim_scores = sim_scores[1:num_recomendaciones+1]

    # Obtener los índices y las similitudes
    movie_indices = [i[0] for i in sim_scores]
    sim_scores_values = [i[1] for i in sim_scores]

    # Verificar que los índices obtenidos están dentro del rango del DataFrame
    valid_indices = [idx for idx in movie_indices if idx in df.index]

    # Crear un DataFrame para mostrar los resultados
    result_df = pd.DataFrame({
        'Índice': valid_indices,
        'Título': df['original_title'].iloc[valid_indices].values,
        'Similitud Coseno': [sim_scores_values[movie_indices.index(idx)] for idx in valid_indices]
    })

    # Imprimir las películas recomendadas con sus índices y similitudes
    #print('\nPelículas recomendadas:')
    

    # Retornar el DataFrame con los resultados
    return result_df

### Crear otras funciones necesarias:

#### Peliculas ya vistas por el ususario

In [9]:
def rated_movies_by_user(user_id, df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]
    
    # Verificar si el usuario tiene películas registradas
    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)

    # Retornar todos los movieId, título y rating como una lista de tuplas
    return list(zip(user_movies_sorted['movieId'], user_movies_sorted['original_title'], user_movies_sorted['rating']))


#### Top peliculas del usuario

In [10]:
def top_movies_by_user(user_id, top_n=3,df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]
    
    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)
    
    # Seleccionar el top N de películas
    top_movies = user_movies_sorted.head(top_n)
    
    # Retornar el movieId y el rating como una lista de tuplas
    return list(zip(top_movies['movieId'], top_movies['original_title'],top_movies['rating']))


#### Recomendador multiple

In [11]:
def recomendador_multiple(movies_list, cosine_sim=cosine_sim_sparseCOUNT, num_recomendaciones=5, df=df_keywords_title):
    # Lista para almacenar los DataFrames de recomendaciones
    all_recommendations = []

    for movie in movies_list:
        movie_id = movie[0]  # ID de la película
        # Crear una Serie que contiene como índice el 'id' y el valor correspondiente al índice de la película
        indices = pd.Series(df.index, index=df['id'])

        # Verificar si el movie_id está en los índices
        if movie_id not in indices.index:
            print(f'ID de película {movie_id} no encontrado.')
            continue

        # Obtener el índice de la película a partir del movie_id
        idx = indices[movie_id]

        # Obtener el título de la película original
        original_title = df.loc[idx, 'original_title']
        #print(f'Título de la película original: {original_title}')
        #print(f'Índice de la película: {movie_id}')

        # Obtener la similitud de coseno para esa película en particular con todas las películas
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordenar la lista de tuplas por la similitud de coseno en orden descendente
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtener las películas más similares ignorando el primero (que es la misma película)
        sim_scores = sim_scores[1:num_recomendaciones + 1]

        # Obtener los índices y las similitudes
        movie_indices = [i[0] for i in sim_scores]
        sim_scores_values = [i[1] for i in sim_scores]

        # Verificar que los índices obtenidos están dentro del rango del DataFrame
        valid_indices = [idx for idx in movie_indices if idx in df.index]

        # Crear un DataFrame para mostrar los resultados
        result_df = pd.DataFrame({
            'Índice': valid_indices,
            'Título': df['original_title'].iloc[valid_indices].values,
            'Similitud Coseno': [sim_scores_values[movie_indices.index(idx)] for idx in valid_indices]
        })

        # Añadir el DataFrame de recomendaciones a la lista
        all_recommendations.append(result_df)

    # Concatenar todos los DataFrames en uno solo
    final_recommendations = pd.concat(all_recommendations, ignore_index=True)

    # Imprimir las películas recomendadas
    #print('\nPelículas recomendadas:')
    #print(final_recommendations)

    # Retornar el DataFrame con los resultados
    return final_recommendations

#### Recomendador por genero

In [6]:
def recomendar_peliculas_por_genero():
    # Pedir al usuario que ingrese un género de película
    genero_usuario = input(
        "Por favor, ingrese un género de película que le interese. Las opciones son:\n"
        "'Thriller', 'Crime', 'Drama', 'Mystery', 'Romance', 'Comedy',\n"
        "'Science Fiction', 'Adventure', 'Family', 'Horror', 'History',\n"
        "'TV Movie', 'Fantasy', 'Action', 'Foreign', 'Music',\n"
        "'Documentary', 'Animation', 'Western', 'War'\n"
    ).lower()

    # Filtrar el DataFrame por el género ingresado, aplicando lower para evitar problemas
    df_filtrado = user_gener_c[user_gener_c['genre'].str.lower().str.contains(genero_usuario)]

    # Calcular métricas globales de las películas dentro del género seleccionado
    df_peliculas = df_filtrado.groupby(['movieId', 'original_title']).agg(
        usuarios_unicos=('userId', 'nunique'),   # Contar valores únicos de 'userId'
        rating_pelicula=('rating', 'sum')        # Sumar el rating total
    ).reset_index()

    total_usuarios = df_peliculas['usuarios_unicos'].sum()
    total_rating = df_peliculas['rating_pelicula'].sum()

    # Calcular las probabilidades sin ponderar y ponderada
    df_peliculas['prob_sin_ponderar'] = (df_peliculas['usuarios_unicos'] / total_usuarios)
    df_peliculas['prob_ponderada'] = (df_peliculas['rating_pelicula'] / total_rating)

    # Ordenar películas por cantidad de usuarios (sin ponderar)
    sorted_peliculas_sin_ponderar = df_peliculas.sort_values(by='prob_sin_ponderar', ascending=False)

    # Tomar las primeras 10 películas sin ponderar
    recomendaciones_sin_ponderar = sorted_peliculas_sin_ponderar.head(10)[['original_title', 'prob_sin_ponderar']]

    # Ordenar películas por rating (ponderado)
    sorted_peliculas_ponderada = df_peliculas.sort_values(by='prob_ponderada', ascending=False)

    # Tomar las primeras 10 películas ponderadas
    recomendaciones_ponderada = sorted_peliculas_ponderada.head(10)[['original_title', 'prob_ponderada']]

    # Concatenar ambas listas de recomendaciones
    df_recomendaciones = pd.concat([recomendaciones_ponderada[['original_title']], recomendaciones_sin_ponderar[['original_title']]])

    # Eliminar duplicados de películas
    df_recomendaciones = df_recomendaciones.drop_duplicates(subset='original_title').reset_index(drop=True)

    # Mostrar el mensaje con las recomendaciones
    print(f"Estas son las películas recomendadas en el género '{genero_usuario}', ¡dale una oportunidad!\n")
    return df_recomendaciones



In [7]:
recomendar_peliculas_por_genero()

AttributeError: Can only use .str accessor with string values!

# Recomendador

In [12]:
def recomendacion_personalizada(df_ratings_peliculas=df_ratings_peliculas, df_keywords_title=df_keywords_title, cosine_sim_sparseCOUNT=cosine_sim_sparseCOUNT):
    # Solicitar el ID del usuario
    user_id = int(input("Por favor, introduce tu ID de usuario: "))
    
    # Obtener las películas mejor valoradas por el usuario
    top_movies = top_movies_by_user(user_id)
    
    if not top_movies:
        return  # Si no hay películas, salir de la función

    # Obtener las recomendaciones basadas en las mejores películas
    recomendaciones = recomendador_multiple(top_movies, cosine_sim=cosine_sim_sparseCOUNT, df=df_keywords_title)

    # Obtener las películas ya vistas por el usuario
    vistas = rated_movies_by_user(user_id)

    # Filtrar las recomendaciones para eliminar las películas vistas
    vistas_ids = {v[0] for v in vistas}  # Obtener solo los movieId de las películas vistas
    recomendaciones_filtradas = recomendaciones[~recomendaciones['Índice'].isin(vistas_ids)]

    # Imprimir las recomendaciones finales
    print('\nAqui tienes unas recomendaciones de peliculas que aun no has visto segun tus ratings anteriores:')
    #print(recomendaciones_filtradas)

    return recomendaciones_filtradas

In [33]:
top_movies_by_user(123678,top_n=10)

[('5991', 'Der letzte Mann', 5.0),
 ('318', 'The Million Dollar Hotel', 5.0),
 ('4886', 'And Then There Were None', 5.0),
 ('1580', 'Rope', 5.0),
 ('1265', 'Bridge to Terabithia', 5.0),
 ('539', 'Psycho', 4.5),
 ('457', 'Sissi', 4.5),
 ('750', 'Murder She Said', 4.5),
 ('8874', "My Best Friend's Wedding", 4.5),
 ('6016', 'The Good Thief', 4.5)]

In [34]:
recomendacion_personalizada()


Aqui tienes unas recomendaciones de peliculas que aun no has visto segun tus ratings anteriores:


,Índice,Título,Similitud Coseno
0,18831,Film,0.420084
1,24890,Tol'able David,0.420084
2,27637,The Scarlet Letter,0.420084
3,31226,Where East is East,0.420084
4,35347,Grandma's Reading Glass,0.420084
5,15291,Down Three Dark Streets,0.480384
6,5615,Playing God,0.470871
7,2191,Wonderland,0.444750
8,4212,Passenger 57,0.444750
9,846,Street Kings,0.419314
